In [2]:
#Gibbs Sampling
#Conergence Test First 
'''
Using Gelman Rubin
if R<1.1 we are good!
W = average STD(parameter) for all chains
B = n/(m-1) sum all M (ave paramj - param ave over all chains)^2
Then R = sqrt(1-1/n+B/W)
'''
import numpy as np
import random, math

def Gelman(p1,p2,p3,p4):
    #take in vector of parameters for *4* Markov chains
    if(len(p1)!=len(p2)!=len(p3)!=len(p4)):
        return("Not valid array inputs")
    n=math.floor(len(p1)/2)
    #split into 8 chains
    p1b = p1[:n]
    p1 = p1[n:]
    p2b = p2[:n]
    p2 = p2[n:]
    p3b = p3[:n]
    p3 = p3[n:]
    p4b = p4[:n]
    p4 = p4[n:]
    p_all = np.matrix([p1,p1b,p2,p2b,p3,p3b,p4,p4b])
    p_ave = p_all.mean()
    chain_std = p_all.std(1)
    W = chain_std.mean()
    B = (n/7) * ((np.mean(p1)-p_ave)**2+(np.mean(p2)-p_ave)**2+
              (np.mean(p3)-p_ave)**2+(np.mean(p4)-p_ave)**2+(np.mean(p1b)-p_ave)**2+(np.mean(p2b)-p_ave)**2+
              (np.mean(p3b)-p_ave)**2+(np.mean(p4b)-p_ave)**2)
    var = (n-1)/n*W + 1/n*B
    R = np.sqrt(var/W)
    return R

'''
Now the hierarchal Gibbs Sampling Problem
Posterior is hard/impossible to sample from,
but conditional posteriors aren't so bad
thetaj~N(thetahatj, Vj)
    where thetahatj = (mu/tau^2+nj*ave(yj)/sig^2)/(1/tau^2+nj/sig^2)
          Vj = 1/(1/tau^2 + nj/sig^2)
mu~N(muhat,tau^2/J)
    Where muhat = ave(thetaj) (all 4 of 'em)
sig^2~Inv-X(n,sighat^2) **to sample Inv-X(v, s^2), sample X = v*s^2/Y
                            where Y ~ Chi-sqr(v)**
    where sighat^2 = 1/n sumJ sumNj (yij - thetaj)^2
tau^2~Inv-X(J-1,tauhat^2)
    where tauhat^2 = 1/J-1 sumJ (thetaj-mu)^2

'''

def theta_sim(i,theta1,theta2,theta3,theta4,mu,sig,tau,y):
    #to simulate theta pass in i (the iterator), thetaj, mu, sig, tau, and data
    #thetaj ~ N(theta hatj, sqrt(vj)
    n1=len(y[0])
    n2=len(y[1])
    n3=len(y[2])
    n4=len(y[3])
    
    #theta hat for each j
    th1 = ((mu[i]/tau[i])+n1*np.mean(y[0])/sig[i])/(1/tau[i]+n1/sig[i])
    th2 = ((mu[i]/tau[i])+n2*np.mean(y[1])/sig[i])/(1/tau[i]+n2/sig[i])
    th3 = ((mu[i]/tau[i])+n3*np.mean(y[2])/sig[i])/(1/tau[i]+n3/sig[i])
    th4 = ((mu[i]/tau[i])+n4*np.mean(y[3])/sig[i])/(1/tau[i]+n4/sig[i])
    #v for each j
    v1 = 1/(1/tau[i]+n1/sig[i])
    v2 = 1/(1/tau[i]+n2/sig[i])
    v3 = 1/(1/tau[i]+n3/sig[i])
    v4 = 1/(1/tau[i]+n4/sig[i])
    #simulate next theta j
    theta1[i+1]=(random.gauss(th1,np.sqrt(v1)))
    theta2[i+1]=(random.gauss(th2,np.sqrt(v2)))
    theta3[i+1]=(random.gauss(th3,np.sqrt(v3)))
    theta4[i+1]=(random.gauss(th4,np.sqrt(v4)))
    return(theta1,theta2,theta3,theta4)

def mu_sim(i,mu,theta1,theta2,theta3,theta4,tau):
    #to simulate mu pass in i, mu, thetaj and tau
    #mu ~ N(muhat,sqrt(tau/J))
    muhat = 1/4*(theta1[i]+theta2[i]+theta3[i]+theta4[i])
    #simulate next mu
    mu[i+1] = random.gauss(muhat,np.sqrt(tau[i]/4))
    return mu
    
def sig_sim(i,y,sig,theta1,theta2,theta3,theta4):
    #to simulate sig pass in i, data, sig, thetaj
    #sig ~ InvChi(n,sighat)
    #**to sample Inv-X(v, s^2), sample X = v*s^2/Y where Y ~ Chi-sqr(v)**
    #calculate sig hat
    sig1 = sum((np.asarray(y[0])-theta1[i])**2)
    sig2 = sum((np.asarray(y[1])-theta2[i])**2)
    sig3 = sum((np.asarray(y[2])-theta3[i])**2)
    sig4 = sum((np.asarray(y[3])-theta4[i])**2)
    n = len(y[0])+len(y[1])+len(y[2])+len(y[3])
    sighat = 1/n*(sig1+sig2+sig3+sig4)
    Y = np.random.chisquare(n)
    #simulate current sig
    sig[i] = n*sighat/Y
    #sig[i]=2
    return sig

#finally
def tau_sim(i,theta1,theta2,theta3,theta4,mu,tau):
    #to simulate tau pass in i, thetaj, mu, and tau
    #tau ~ InvChi(J-1,tauhat)
    #calculate tau hat
    tau1 = (theta1[i]-mu[i])**2
    tau2 = (theta2[i]-mu[i])**2
    tau3 = (theta3[i]-mu[i])**2
    tau4 = (theta4[i]-mu[i])**2
    tauhat = 1/3*(tau1+tau2+tau3+tau4)
    Y = np.random.chisquare(3)
    #simulate current tau
    tau[i] = 3*tauhat/Y
    #tau[i] = 2
    return tau

import random,math
 
def gibbs(y):
    N=100001
    burn=50000
    i = 0
    '''
    initialize paramaters
    '''
    theta1=np.zeros(N)
    theta2=np.zeros(N)
    theta3=np.zeros(N)
    theta4=np.zeros(N)
    mu=np.zeros(N)
    sig=np.zeros(N)
    tau=np.zeros(N)
    
    theta1[i]=random.gauss(60,10)
    theta2[i]=random.gauss(60,10)
    theta3[i]=random.gauss(60,10)
    theta4[i]=random.gauss(60,10)
    mu[i]=random.gauss(60,10)
    '''
    def sig(i,y,sig,theta1,theta2,theta3,theta4):
    #to simulate sig pass in i, data, sig, thetaj
    '''
    sig = sig_sim(i,y,sig,theta1,theta2,theta3,theta4)
    
    '''
    def tau(i,theta1,theta2,theta3,theta4,mu,tau):
    #to simulate tau pass in i, thetaj, mu, and tau
    '''
    tau = tau_sim(i,theta1,theta2,theta3,theta4,mu,tau)
    print(mu[i],sig[i],tau[i],theta1[i],theta2[i],theta3[i],theta4[i])
    
    while i < N-1:
        '''
        def theta(i,theta1,theta2,theta3,theta4,mu,sig,tau,y):
        #to simulate theta pass in i (the iterator), thetaj, mu, sig, tau, and data
        '''
        theta1,theta2,theta3,theta4 = theta_sim(i,theta1,theta2,theta3,theta4,mu,sig,tau,y)
        '''
        def mu(i,mu,theta1,theta2,theta3,theta4,tau):
        #to simulate mu pass in i, mu, thetaj and tau
        '''
        mu = mu_sim(i,mu,theta1,theta2,theta3,theta4,tau)
        i+=1
        sig = sig_sim(i,y,sig,theta1,theta2,theta3,theta4)
        tau = tau_sim(i,theta1,theta2,theta3,theta4,mu,tau)
    print(mu[i],sig[i],tau[i],theta1[i],theta2[i],theta3[i],theta4[i])
        
    return(mu[burn:N-1],sig[burn:N-1],tau[burn:N-1],theta1[burn:N-1],theta2[burn:N-1],theta3[burn:N-1],theta4[burn:N-1])

#data
#unusual but make A,B,C,D row 1,2,3,4
y = [[62,60,63,59],[63,67,71,64,65,66],
             [68,66,71,67,68,68],
             [56,62,60,61]]
muA,sigA,tauA,theta1A,theta2A,theta3A,theta4A = gibbs(y)
muB,sigB,tauB,theta1B,theta2B,theta3B,theta4B = gibbs(y)
muC,sigC,tauC,theta1C,theta2C,theta3C,theta4C = gibbs(y)
muD,sigD,tauD,theta1D,theta2D,theta3D,theta4D = gibbs(y)
'''
Now run Gelman
def Gelman(p1,p2,p3,p4):
'''
R = Gelman(muA,muB,muC,muD)
print(R)

        

48.0378135121 425.741918103 321.469787286 64.4816284534 52.9589922223 31.0714783723 55.1453225189
74.1389420385 4.6828697886 72.8107449031 61.5283683981 63.8597504538 68.7542485068 60.8354075522
74.3766041714 123.720559167 118.458133739 71.5585387654 52.5824013357 63.7807622709 68.2401453046
61.1340995077 4.14848135908 28.8245375355 62.5785043436 65.6323067548 67.6751954641 59.0211686143
71.0231852227 84.6789936331 153.667630062 57.3622133188 68.2850613706 52.1662682284 59.8262577492
62.3895645338 14.0358419106 4.37261303687 58.4698221293 63.538339831 67.8131096398 63.3453264666
52.6282387192 120.944476808 415.589953792 80.5762373028 57.4429460442 64.4804052102 68.5609643194
64.1116903337 4.27442390265 12.3331335584 62.5637347678 66.9905525405 68.2402138099 60.4184403545
3.20617147756


In [10]:
import numpy as np
import random, math

def Gelman(p1,p2,p3,p4):
    #take in vector of parameters for *4* Markov chains
    if(len(p1)!=len(p2)!=len(p3)!=len(p4)):
        return("Not valid array inputs")
    n=math.floor(len(p1)/2)
    #split into 8 chains
    p1b = p1[:n]
    p1 = p1[n:]
    p2b = p2[:n]
    p2 = p2[n:]
    p3b = p3[:n]
    p3 = p3[n:]
    p4b = p4[:n]
    p4 = p4[n:]
    p_all = np.matrix([p1,p1b,p2,p2b,p3,p3b,p4,p4b])
    p_ave = p_all.mean()
    chain_std = p_all.std(1)
    W = chain_std.mean()
    B = (n/7) * ((np.mean(p1)-p_ave)**2+(np.mean(p2)-p_ave)**2+
              (np.mean(p3)-p_ave)**2+(np.mean(p4)-p_ave)**2+(np.mean(p1b)-p_ave)**2+(np.mean(p2b)-p_ave)**2+
              (np.mean(p3b)-p_ave)**2+(np.mean(p4b)-p_ave)**2)
    var = (n-1)/n*W + 1/n*B
    R = np.sqrt(var/W)
    return R
R = Gelman(tauA,tauB,tauC,tauD)
#numpy.percentile(a, q,
print("tau**2: ",np.percentile(tauA,2.5)," ",np.percentile(tauA,25)," ",
      np.percentile(tauA,50)," ",np.percentile(tauA,75)," ",
      np.percentile(tauA,97.5)," ",R)

tau**2:  5.01890372494   15.7774929556   33.1965156999   83.0472695656   918.337253101   1.2420671464


In [29]:
Y = np.random.chisquare(3)
Y

2.5328180290202833